# Simple Linear Regression

En este cuaderno veremos los conceptos detrás de una regresión lineal simple: `Y = b0 + b1*X`

* Intercepto `b0`
* Coeficiente `b1`
* Regresor `X`
* Objetivo `y`

También veremos el concepto de OLS aplicado como medida de rendimiento para este algoritmo. Midiendo el error entre el valor de salida predicho por el modelo y el valor original para cada instancia de X, podremos mejorar el entrenamiento.

La regresión linear se basa en cuatro suposiciones:
1. Linearidad: significa que la variable dependiente tiene una relación lineal con la variable independiente.
2. Normalidad: significa que los errores de las observaciones estan normalmente distribuidos.
3. Independencia: significa que el error de cada observación es independiente de las demás.
4. Homoscedasticidad: significa que la varianza del error de cada observación es constante para todas las observaciones.
5. Baja multi-colinearidad: significa que las variables independientes no estan altamente correlacionadas entre si.


## 1. Creamos un set de datos

En este caso a modo de ejemplo vamos a `crear` datos de 3 formas distintas:

1. `np.arange`
2. `np.random.uniform`
3. `np.random.randn`

La idea es que vean las diferencias entre las formas que posee NumPy. Para investigar cada método (ver que argumentos lleva y que retorna) pueden pararse dentro de los parentesis de cualquier función y usar la combinación `shift + ctrl + spacebar`

In [ ]:
import numpy as np

In [ ]:
np.random.seed(42) # Esta linea es para conservar los mismos valores "random" cada vez que se ejecute la celda
X = np.arange(0,300,dtype=int) #Empieza en 0 hasta el valor de stop que se elija, todos los valores son del tipo 'int'
X_uniform = np.random.uniform(0,1,size=300)
X_randn = np.random.randn(300)

In [ ]:
type(X), type(X_uniform), type(X_randn)

In [ ]:
print(f"X values samples: {X[:5]}, value max: {X.max()}, value min: {X.min()}, mean: {X.mean()}, std: {X.std():.2f}")
print(f"X_uniform values samples: {X_uniform[:5]}, value max: {X_uniform.max()}, value min: {X_uniform.min()}, mean: {X_uniform.mean():.5f}, std: {X_uniform.std():.2f}")
print(f"X_randn values samples: {X_randn[:5]}, value max: {X_randn.max()}, value min:{X_randn.min()}, mean: {X_randn.mean():.5f}, std: {X_randn.std():.2f}")

## 2. Visualizamos

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Visualizamos la distribución de los datos/valores de las variables independientes creadas
plt.figure(figsize=(10,7))
plt.subplot(1,3,1)
plt.hist(X,density=True)
plt.title("Histograma de X")
plt.subplot(1,3,2)
plt.hist(X_uniform,density=True)
plt.title("Histograma de X_uniform")
plt.subplot(1,3,3)
plt.hist(X_randn,density=True)
plt.title("Histograma de X_randn");

## 3. Armamos una relación lineal

In [ ]:
# Vamos a crear una relación entre una variable dependiente y las variables independientes creadas hasta ahora
# Usaremos para este ejemplo X y X_uniform
# La relación es lineal del tipo: y = X*coef + intercept

coef = 0.5 #beta1
intercept = -1.3 #beta0
y = (X*coef) + intercept #En esta relación usamos como variable independiente el array X creado con np.arange()
y_uniform = (X_uniform*coef) + intercept #En esta relación usamos como variable independiente el array X_uniform creado con np.random.uniform()
y_randn = (X_randn*coef) + intercept

In [ ]:
type(y), type(y_uniform), type(y_randn)

## 4. Visualizamos

In [ ]:
# Visualizamos la distribución de los datos/valores de las variables dependientes creadas
plt.figure(figsize=(10,7))
plt.subplot(1,3,1)
plt.hist(X,density=True)
plt.title("Histograma de y")
plt.subplot(1,3,2)
plt.hist(y_uniform,density=True)
plt.title("Histograma de y_uniform")
plt.subplot(1,3,3)
plt.hist(y_randn,density=True)
plt.title("Histograma de y_randn");

In [ ]:
#Visualizamos ambas relaciones lineales
plt.figure(figsize=(10,7))
plt.subplot(1,3,1)
plt.scatter(X,y,c='green',s=4)
plt.title("Linear X,y")
plt.subplot(1,3,2)
plt.scatter(X_uniform,y_uniform,c='purple',s=4)
plt.title("Linear X_uniform,y_uniform")
plt.subplot(1,3,3)
plt.scatter(X_randn,y_randn,c='lime',s=4)
plt.title("Linear X_randn,y_randn");

## 5. Armamos datos de entrenamiento y testeo

Para avanzar en la práctica de regresión lineal tomaremos el dataset de `X,y`.
Los pasos para implementar una regresión son:
1. Preprocesar los datos (si es necesario normalizar o estandarizar).
2. Separar los datos en conjuntos de `train`, `test` y `validation`.
3. Elegimos el modelo a utilizar.
4. Entrenamos el modelo.
5. Evaluamos el modelo.
6. Ajustamos el modelo (en caso de ser necesario).
7. Repetimos el paso 5 y 6 hasta lograr métricas que nos satisfagan.
8. Realizamos predicciones con datos nuevos.

In [ ]:
#Primero separamos de forma tradicional
split_size = int(len(X)*0.8)
print(f"Tamaño del set de entrenamiento: {split_size}\nTamaño del set de testeo: {len(X)-split_size}")

In [ ]:
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
print(f"X_train length: {len(X_train)}, y_train length: {len(y_train)}")
print(f"X_test length: {len(X_test)}, y_test length: {len(y_test)}")

In [ ]:
#Visualizamos los datos de entrenamiento y testeo
plt.figure(figsize=(10,7))
plt.subplot(1,2,1)
plt.scatter(X,y,c='purple',s=2)
plt.title("Full dataset")
plt.subplot(1,2,2)
plt.scatter(X_train,y_train,c='orange',s=4,label='Train set')
plt.scatter(X_test,y_test,c='green',s=4,label='Test set')
plt.title("Linear splited set")
plt.legend();

>**Nota:** podemos ver que el set original de datos `X,y` ahora está dividido en conjunto de entrenamiento `[X_train, y_train]` y de testeo `[X_test, y_test]` de forma **no aleatoria** lo cual en este caso el set de testeo es representativo de la forma de la función original `y = b0 + b1*w` pero en otros casos como una función `sigmoide` esto no funcionaría porque separar los datos de esta forma **seguramente** generaría un set de datos de testo **no representativos** del resto de datos.

### 5.1 Creamos un set de datos no lineales (SIGMOID)



In [ ]:
# Creamos una relación con la función sigmoide implementada con numpy
x = np.linspace(-10, 10, 300)
z = 1/(1 + np.exp(-x))
print(f"Tipo de estructura de datos: {type(z)}")

In [ ]:
# Separamos el conjunto de datos X,z en train y test
X_train_sigm, y_train_sigm = x[:split_size], z[:split_size]
X_test_sigm, y_test_sigm = x[split_size:], z[split_size:]
print(f"X_train length: {len(X_train_sigm)}, y_train length: {len(y_train_sigm)}")
print(f"X_test length: {len(X_test_sigm)}, y_test length: {len(y_test_sigm)}")

In [ ]:
#Visualizamos los datos de entrenamiento y testeo no lineales
plt.figure(figsize=(10,7))
plt.subplot(1,2,1)
plt.scatter(x,z,c='purple',s=2)
plt.title("Full dataset Sigmoid")
plt.subplot(1,2,2)
plt.scatter(X_train_sigm,y_train_sigm,c='orange',s=4,label='Train set')
plt.scatter(X_test_sigm,y_test_sigm,c='green',s=4,label='Test set')
plt.title("Sigmoid splited set")
plt.legend();

>**Nota:** Dado el ejemplo de arriba 👆 vemos que hay que buscar una alternativa para separar los datos de forma aleatoria, así poder conservar la forma de la relación en los datos de testeo.

## 6. Separamos datos de forma randomizada

In [ ]:
#para esto importamos scikit-learn que posee una función para separar los datos de forma ordenada
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Xtrain length: {len(Xtrain)}, ytrain length: {len(ytrain)}")
print(f"Xtest length: {len(Xtest)}, ytest length: {len(ytest)}")

In [ ]:
#Visualizamos los datos de entrenamiento y testeo
plt.figure(figsize=(12,7))
plt.subplot(1,2,1)
plt.scatter(X,y,c='purple',s=2)
plt.title("Full dataset")
plt.subplot(1,2,2)
plt.scatter(Xtrain,ytrain,c='orange',s=4,label='Train set')
plt.scatter(Xtest,ytest,c='green',s=6,label='Test set')
plt.title("Randomize splited set")
plt.legend();

In [ ]:
# Creamos una relación con la función sigmoide implementada con numpy
x_sigmoid = np.linspace(-10, 10, 300)
z = 1/(1 + np.exp(-x_sigmoid))
Xtrain_sigmoid, Xtest_sigmoid, ytrain_sigmoid, ytest_sigmoid = train_test_split(x_sigmoid, z, test_size=0.2, random_state=42)

In [ ]:
#Visualizamos los datos de entrenamiento y testeo no lineales
plt.figure(figsize=(10,7))
plt.subplot(1,2,1)
plt.scatter(x_sigmoid,z,c='purple',s=2)
plt.title("Full dataset Sigmoid")
plt.subplot(1,2,2)
# plt.scatter(Xtrain_sigmoid,ytrain_sigmoid,c='orange',s=4,label='Train set')
plt.scatter(Xtest_sigmoid,ytest_sigmoid,c='green',s=4,label='Test set')
plt.title("Sigmoid splited set")
plt.legend();

## 7. Elegimos el modelo

En este caso vamos a probar 3 modelos de regresión, las tres estan basadas en el modelo OLS (mínimos cuadrados ordinarios).

1. Linear Regression (OLS)
2. Lasso: basada en OLS pero incorpora un peso `Lambda` para castigar aquellos valores demasiado altos que nos pueden conducir a overfitting y un poco entendimiento de la verdadera relación entre las variables predictoras y el objetivo.
3. Ridge: basado en OLS y tambien incorpora una forma de calculo para castigar los valos muy altos y reducir notablemente el overfitting. Sin embargo, no realiza una selección de variables como es el caso de Lasso.

El entrenamiento de los modelos lo haremos con el método `fit()` de cada objeto modelo instanciado. Usaremos el conjunto `[Xtrain,ytrain]` para entrenar y luego evaluaremos los modelos con el conjunto `[Xtest,ytest]`.

In [ ]:
#Scikit-learn ya trae implementados muchos modelos, cuya hiper-parametrización resulta sencilla de implementar
from sklearn.linear_model import  LinearRegression, Lasso, Ridge
linear_r = LinearRegression()
lasso_r = Lasso()
ridge_r = Ridge()

In [ ]:
Xtrain.reshape(-1,1).shape

In [ ]:
#entrenamos con la función .fit() y usamos el conjunto de entrenamiento solo para entrenar
linear_r.fit(Xtrain.reshape(-1,1),ytrain.reshape(-1,1))
lasso_r.fit(Xtrain.reshape(-1,1),ytrain.reshape(-1,1))
ridge_r.fit(Xtrain.reshape(-1,1),ytrain.reshape(-1,1))

## 8. Evaluamos el modelo

Para esto usamos el método `.score()` para calcular la metrica *coeficiente de determinación* o **R2** que incluye scikit-learn.

Usaremos el conjunto de testeo `[Xtest,ytest]` para calcular este score.

In [ ]:
#Primero evaluamos los modelos entrenados
print(f"Linear Regression accuracy: {linear_r.score(Xtest.reshape(-1,1),ytest.reshape(-1,1))}")
print(f"Lasso Regression accuracy: {lasso_r.score(Xtest.reshape(-1,1),ytest.reshape(-1,1))}")
print(f"Ridge Regression accuracy: {ridge_r.score(Xtest.reshape(-1,1),ytest.reshape(-1,1))}")

## 9. Hacemos predicciones

Para hacer predicciones sobre el conjunto de testeo `[Xtest,ytest]`, ya que es el unico que no se uso durante el entrenamiento, sería como poner a prueba el modelo con datos que nunca observó hasta ahora.

También calcularemos las siguientes métricas:
1. R squared score.
2. Mean average error.
3. Mean squared error.

Las metricas usadas son unas pocas de varias que ofrece scikit-learn https://scikit-learn.org/stable/modules/classes.html#regression-metrics.


In [ ]:
from sklearn import metrics as ms
def calculate_metrics(y_true:None,y_pred:None):
  """Esta funcion calcula accuracy,mae,mse y retorna un diccionario con dichos valores"""
  r2 = ms.r2_score(y_true=y_true,y_pred=y_pred)
  mae = ms.mean_absolute_error(y_true=y_true,y_pred=y_pred)
  mse = ms.mean_squared_error(y_true=y_true,y_pred=y_pred)
  return {"R2":r2,"MAE":mae,"MSE":mse}

In [ ]:
#Primero calculamos las metricas LinearRegression
y_pred = linear_r.predict(Xtest.reshape(-1,1))
linear_r_metrics = calculate_metrics(y_true=ytest.reshape(-1,1), y_pred=y_pred)
linear_r_metrics

In [ ]:
#Segundo calculamos las metricas Lasso
y_pred = lasso_r.predict(Xtest.reshape(-1,1))
lasso_r_metrics = calculate_metrics(y_true=ytest.reshape(-1,1), y_pred=y_pred)
lasso_r_metrics

In [ ]:
#Tercero calculamos las metricas Ridge
y_pred = ridge_r.predict(Xtest.reshape(-1,1))
ridge_r_metrics = calculate_metrics(y_true=ytest.reshape(-1,1), y_pred=y_pred)
ridge_r_metrics

In [ ]:
import pandas as pd
pd.DataFrame({"Linear":linear_r_metrics,
              "Lasso":lasso_r_metrics,
              "Ridge":ridge_r_metrics})

In [ ]:
pd.DataFrame({"Linear":linear_r_metrics,
              "Lasso":lasso_r_metrics,
              "Ridge":ridge_r_metrics}).T[['MSE']].plot(kind='bar');

## 10. Diagnostico del modelo

In [ ]:
ytest.shape, y_pred.shape

In [ ]:
#Evaluamos el supuesto de linealidad de los residuos
from statsmodels.stats.stattools import durbin_watson
residuals = ytest - np.squeeze(y_pred)
dw = durbin_watson(residuals)
print(f"Durbin Watson must be between [0,4] where both extremes are residual correlated and the middle 2 means independency between errors:\n{dw}")

In [ ]:
#Evaluamos el supuesto de homoscedasticidad
plt.scatter(y_pred,residuals)
plt.xlabel('Predicted')
plt.ylabel('Error')
plt.title('Homocedasticity');

In [ ]:
#Evaluamos la normalidad
plt.hist(residuals)

In [ ]:
#Tambien podemos evaluar la linealidad de los residuos por quantiles (qqnorm)
import statsmodels.api as sm
import pylab
sm.qqplot(residuals,line='q')
pylab.show()



---



>🛠 **Ejercicio**: crear un conjunto de datos para regression con `from sklearn.datasets import make_regression` con una sola feature, y si quieren le agrega noise (5)

pd: Sigan los pasos del notebook